## Comodo Exmaple 
This examples, load a basic robot model (i.e. composed only of basic shapes), modifies the links of such a robot model by elongating the legs, define instances of the TSID (Task Based Inverse Dynamics) and Centroidal MPC  controller and simulate the behavior of the robot using drake.  

In [1]:
# Comodo import
# from comodo.mujocoSimulator.mujocoSimulator import MujocoSimulator
# from comodo.drakeSimulator.drakeSimulator import DrakeSimulator
# from comodo.robotModel.robotModel import RobotModel
# from comodo.robotModel.createUrdf import createUrdf
# from comodo.centroidalMPC.centroidalMPC import CentroidalMPC
# from comodo.centroidalMPC.mpcParameterTuning import MPCParameterTuning
# from comodo.TSIDController.TSIDParameterTuning import TSIDParameterTuning
# from comodo.TSIDController.TSIDController import TSIDController

import sys
sys.path.append('/home/laniakea/git/odio_urdf/')
from odio_urdf import *

sys.path.append('../src')
from comodo.mujocoSimulator.mujocoSimulator import MujocoSimulator
from comodo.drakeSimulator.drakeSimulator import DrakeSimulator
from comodo.robotModel.robotModel import RobotModel
from comodo.robotModel.createUrdf import createUrdf
from comodo.centroidalMPC.centroidalMPC import CentroidalMPC
from comodo.centroidalMPC.mpcParameterTuning import MPCParameterTuning
from comodo.TSIDController.TSIDParameterTuning import TSIDParameterTuning
from comodo.TSIDController.TSIDController import TSIDController

# additional depenedencies
import resolve_robotics_uri_py
import time
from contextlib import redirect_stderr, redirect_stdout
import gc
import os
import pygmo as pg
from datetime import datetime
import pickle


In [2]:
# General  import 
import xml.etree.ElementTree as ET
import numpy as np
import tempfile
import urllib.request
import logging

In [3]:
# set the logging level
logging.getLogger().setLevel(logging.CRITICAL)
logging.getLogger().propagate = False


In [4]:
robot_model = "stickbot"

if robot_model == "stickbot":
    # Getting stickbot urdf file and convert it to string 
    urdf_robot_file = tempfile.NamedTemporaryFile(mode="w+")
    url = 'https://raw.githubusercontent.com/icub-tech-iit/ergocub-gazebo-simulations/master/models/stickBot/model.urdf'
    urllib.request.urlretrieve(url, urdf_robot_file.name)
    # Load the URDF file
    tree = ET.parse(urdf_robot_file.name)
    root = tree.getroot()

    # Convert the XML tree to a string
    robot_urdf_string_original = ET.tostring(root)

    create_urdf_instance = createUrdf(
        original_urdf_path=urdf_robot_file.name, save_gazebo_plugin=False
    )

if robot_model == "ergoCub":
    # Getting ergoCub urdf file and convert it to string 
    urdf_robot_file = resolve_robotics_uri_py.resolve_robotics_uri("package://ergoCub/robots/ergoCubSN000/model.urdf")
    
    # Load the URDF file
    tree = ET.parse(urdf_robot_file)
    root = tree.getroot()

    # Convert the XML tree to a string
    robot_urdf_string_original = ET.tostring(root)

    create_urdf_instance = createUrdf(
        original_urdf_path=urdf_robot_file, save_gazebo_plugin=False
    )

In [5]:
# Define parametric links and controlled joints  
legs_link_names = ["hip_3", "lower_leg"]
joint_name_list = [
    "r_shoulder_pitch",
    "r_shoulder_roll",
    "r_shoulder_yaw",
    "r_elbow",
    "l_shoulder_pitch",
    "l_shoulder_roll",
    "l_shoulder_yaw",
    "l_elbow",
    "r_hip_pitch",
    "r_hip_roll",
    "r_hip_yaw",
    "r_knee",
    "r_ankle_pitch",
    "r_ankle_roll",
    "l_hip_pitch",
    "l_hip_roll",
    "l_hip_yaw",
    "l_knee",
    "l_ankle_pitch",
    "l_ankle_roll",
]

In [6]:
# Define the robot modifications
modifications = {}
for item in legs_link_names:
    left_leg_item = "l_" + item
    right_leg_item = "r_" + item
    modifications.update({left_leg_item: 1.2})
    modifications.update({right_leg_item: 1.2})
# Motors Parameters 
Im_arms = 1e-3*np.ones(4) # from 0-4
Im_legs = 1e-3*np.ones(6) # from 5-10
kv_arms = 0.001*np.ones(4) # from 11-14
kv_legs = 0.001*np.ones(6) #from 20

Im = np.concatenate((Im_arms, Im_arms, Im_legs, Im_legs))
kv = np.concatenate((kv_arms, kv_arms, kv_legs, kv_legs))

In [7]:
# Modify the robot model and initialize
create_urdf_instance.modify_lengths(modifications)
urdf_robot_string = create_urdf_instance.write_urdf_to_file()
create_urdf_instance.reset_modifications()
robot_model_init = RobotModel(urdf_robot_string, "stickBot", joint_name_list)
s_des, xyz_rpy, H_b = robot_model_init.compute_desired_position_walking()

In [8]:
# Define simulator and initial position
drake_instance = DrakeSimulator()
drake_instance.set_visualize_robot_flag(True)
drake_instance.load_model(robot_model_init, s=s_des, xyz_rpy=xyz_rpy, kv_motors=kv, Im=Im)
s, ds, tau = drake_instance.get_state()
t = drake_instance.get_simulation_time()
H_b = drake_instance.get_base()
# beware of the velocity ordering
w_b = drake_instance.get_base_velocity()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [11]:
# Define the controller parameters  and instantiate the controller
# Controller Parameters
tsid_parameter = TSIDParameterTuning()
mpc_parameters = MPCParameterTuning()

x_k = np.array([   0.        ,  405.13438589, 1000.        ,  278.40788035,
        408.4593702 ,  743.86629237,  285.3510762 ,    0.        ,
        126.93881708,  818.23849541,  970.11706261,  635.60135141,
        802.46857797,  180.83407013,  233.28060849,    0.        ,
        736.74425248,    0.        ,  899.7710529 ,  658.72333256,
        606.20333008,    0.        , 1000.        ,   17.59620738,
          0.        , 1000.        ,    0.        ,   96.71097781])

tsid_parameter.set_from_x_k(x_k)

# TSID Instance
TSID_controller_instance = TSIDController(frequency=0.01, robot_model=robot_model_init)
TSID_controller_instance.define_tasks(tsid_parameter)
TSID_controller_instance.set_state_with_base(s, ds, H_b, w_b, t)

# MPC Instance
step_lenght = 0.1
mpc = CentroidalMPC(robot_model=robot_model_init, step_length=step_lenght)
mpc.intialize_mpc(mpc_parameters=mpc_parameters)

# Set desired quantities
mpc.configure(s_init=s_des, H_b_init=H_b)
TSID_controller_instance.compute_com_position()
mpc.define_test_com_traj(TSID_controller_instance.COM.toNumPy())

# TODO: Set initial robot state and plan trajectories 
drake_instance.step(1)

# Reading the state 
s, ds, tau = drake_instance.get_state()
H_b = drake_instance.get_base()
w_b = drake_instance.get_base_velocity()
t = drake_instance.get_simulation_time()

# MPC
mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
mpc.initialize_centroidal_integrator(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
mpc_output = mpc.plan_trajectory()

In [12]:
# Set loop variables 
TIME_TH = 20.0

# Define number of steps
n_step = int(
    TSID_controller_instance.frequency / drake_instance.get_simulation_frequency()
)
n_step_mpc_tsid = int(mpc.get_frequency_seconds() / TSID_controller_instance.frequency)

counter = 0
mpc_success = True
energy_tot = 0.0
succeded_controller = True

In [13]:
# Simulation-control loop 
drake_instance.meshcat.StartRecording()
while t < TIME_TH:
    
    # Reading robot state from simulator
    s, ds, tau = drake_instance.get_state()
    energy_i = np.linalg.norm(tau)
    H_b = drake_instance.get_base()
    w_b = drake_instance.get_base_velocity()
    t = drake_instance.get_simulation_time()

    # Update TSID
    TSID_controller_instance.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)

    # MPC plan 
    if counter == 0:
        mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
        mpc.update_references()
        mpc_success = mpc.plan_trajectory()
        mpc.contact_planner.advance_swing_foot_planner()
        if not (mpc_success):
            logging.error("MPC failed")
            break

    # Reading new references
    com, dcom, forces_left, forces_right, ang_mom = mpc.get_references()
    left_foot, right_foot = mpc.contact_planner.get_references_swing_foot_planner()

    # Update references TSID 
    TSID_controller_instance.update_task_references_mpc(
        com=com,
        dcom=dcom,
        ddcom=np.zeros(3),
        left_foot_desired=left_foot,
        right_foot_desired=right_foot,
        s_desired=np.array(s_des),
        wrenches_left=forces_left,
        wrenches_right=forces_right,
        ang_mom_desired=ang_mom,
    )

    # Run control 
    succeded_controller = TSID_controller_instance.run()

    if not (succeded_controller):
        logging.error("Controller failed")
        break

    tau = TSID_controller_instance.get_torque()

    # Step the simulator
    drake_instance.set_input(tau)
    # drake_instance.step_with_motors(n_step=n_step, torque=tau)
    try:
        drake_instance.step(n_step=n_step)
    except RuntimeError as e:
        logging.error(e)
        break
    counter = counter + 1

    if counter == n_step_mpc_tsid:
        counter = 0
drake_instance.meshcat.StopRecording()
drake_instance.meshcat.PublishRecording()

In [12]:
# Closing visualization
drake_instance.close_visualization()

In [13]:
# create a function to optimise via pygmo
from IPython.display import clear_output

def cost_function(x_k):
    # bringing in robot_model_init to prevent pyswig
    with redirect_stderr(open(os.devnull, "w")):
        robot_model_init = RobotModel(urdf_robot_string, "stickBot", joint_name_list)
        s_des, xyz_rpy, H_b = robot_model_init.compute_desired_position_walking()
    # drake instance
    drake_instance = DrakeSimulator()
    drake_instance.set_visualize_robot_flag(False)
    drake_instance.load_model(robot_model_init, s=s_des, xyz_rpy=xyz_rpy, kv_motors=kv, Im=Im)
    # initialise the system
    s, ds, tau = drake_instance.get_state()
    t = drake_instance.get_simulation_time()
    H_b = drake_instance.get_base()
    # beware of the velocity ordering
    w_b = drake_instance.get_base_velocity()

    tsid_parameter = TSIDParameterTuning()
    mpc_parameters = MPCParameterTuning()
    tsid_parameter.set_from_x_k(x_k)

    # TSID Instance
    TSID_controller_instance = TSIDController(frequency=0.01, robot_model=robot_model_init)
    TSID_controller_instance.define_tasks(tsid_parameter)
    TSID_controller_instance.set_state_with_base(s, ds, H_b, w_b, t)

    # MPC Instance
    step_lenght = 0.1
    mpc = CentroidalMPC(robot_model=robot_model_init, step_length=step_lenght)
    mpc.intialize_mpc(mpc_parameters=mpc_parameters)

    # Set desired quantities
    mpc.configure(s_init=s_des, H_b_init=H_b)
    TSID_controller_instance.compute_com_position()
    mpc.define_test_com_traj(TSID_controller_instance.COM.toNumPy())

    # TODO: Set initial robot state and plan trajectories 
    drake_instance.step(1)

    # Reading the state 
    s, ds, tau = drake_instance.get_state()
    H_b = drake_instance.get_base()
    w_b = drake_instance.get_base_velocity()
    t = drake_instance.get_simulation_time()

    # MPC
    mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
    mpc.initialize_centroidal_integrator(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
    mpc_output = mpc.plan_trajectory()

    # Set loop variables 
    TIME_TH = 20.0

    # Define number of steps
    n_step = int(
        TSID_controller_instance.frequency / drake_instance.get_simulation_frequency()
    )
    n_step_mpc_tsid = int(mpc.get_frequency_seconds() / TSID_controller_instance.frequency)

    counter = 0
    mpc_success = True
    energy_tot = 0.0
    succeded_controller = True

    # Simulation-control loop 
    while t < TIME_TH:
        
        # Reading robot state from simulator
        s, ds, tau = drake_instance.get_state()
        energy_i = np.linalg.norm(tau)
        try:
            H_b = drake_instance.get_base()
        except RuntimeError as e:
            return drake_instance.get_simulation_time()
        w_b = drake_instance.get_base_velocity()
        t = drake_instance.get_simulation_time()

        # Update TSID
        TSID_controller_instance.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)

        # MPC plan 
        if counter == 0:
            # print('in mpc')
            mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
            mpc.update_references()
            # print('plan')
            mpc_success = mpc.plan_trajectory()
            # print('advance foot planner')
            mpc.contact_planner.advance_swing_foot_planner()
            # if not (mpc_success):
            #     logging.error("MPC failed")
            #     break

        # Reading new references
        # print('get references')
        com, dcom, forces_left, forces_right, ang_mom = mpc.get_references()
        left_foot, right_foot = mpc.contact_planner.get_references_swing_foot_planner()

        # Update references TSID 
        TSID_controller_instance.update_task_references_mpc(
            com=com,
            dcom=dcom,
            ddcom=np.zeros(3),
            left_foot_desired=left_foot,
            right_foot_desired=right_foot,
            s_desired=np.array(s_des),
            wrenches_left=forces_left,
            wrenches_right=forces_right,
            ang_mom_desired=ang_mom,
        )

        # Run control
        # print('run tsid')
        # with redirect_stderr(open(os.devnull, "w")):
        succeded_controller = TSID_controller_instance.run()

        if not (succeded_controller):
            # logging.error("Controller failed")
            return drake_instance.get_simulation_time()

        tau = TSID_controller_instance.get_torque()
        # print('got torque?')
        # Step the simulator
        drake_instance.set_input(tau)
        # drake_instance.step_with_motors(n_step=n_step, torque=tau)
        try:
            drake_instance.step(n_step=n_step)
        except RuntimeError as e:
            # logging.error(e)
            return drake_instance.get_simulation_time()
            
        counter = counter + 1
        # print('stepping')

        if counter == n_step_mpc_tsid:
            counter = 0
        # clear_output(wait=True)
    return 0

In [14]:
%%time
x_k = x_k = np.array([684.25282492, 648.05228734, 569.96763936, 350.65899653,
       949.98434623, 515.09198331, 845.83335962,  17.95951342,
       314.5705753 , 676.403354  , 440.72680377, 467.56525625,
       681.69608946,  97.4551466 , 715.55867788, 416.80935819,
       470.72979393, 680.49815841, 876.67111456, 654.65448001,
       329.59585922,  65.34583639, 975.68181074, 342.1368993 ,
       237.81553041, 606.40982421,  32.24741521,  10.83276846])


# with redirect_stderr(None):
tf = cost_function(x_k)
print(tf)

ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.
1.1910000000000007
CPU times: user 8.64 s, sys: 18.2 s, total: 26.9 s
Wall time: 2.28 s


[OsqpEigen::Solver::initSolver] Unable to setup the workspace.
[OsqpEigen::Solver::updateHessianMatrix] Unable to Initialize the solver.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.


In [15]:
class GainTuningProblem:
    # @staticmethod
    def fitness(self, x):
        # with redirect_stdout(open(os.devnull, "w")):
        result = -cost_function(x)
        gc.collect()
        return [result]

    def get_bounds(self):
        return ([0] * 28, [1000] * 28)

In [16]:
%%time

prob = pg.problem(GainTuningProblem())
algo = pg.algorithm(pg.cmaes(gen=10, force_bounds=True, seed=0))
algo.set_verbosity(1)
archi = pg.archipelago(
    n=5,
    algo=algo,
    prob=prob,
    pop_size=10,
)
archi.evolve()
print(archi)
ret_flag = archi.wait_check()
res = archi.get_champions_f()
x = archi.get_champions_x()
print(archi, ret_flag)

dict_log = {}
dict_log["res"] = res
dict_log["x"] = x

filename = "../optimisation_logs/{}.pkl".format(
    datetime.now().strftime("%d%m%Y%H%M%S")
)
with open(filename, "wb") as f:
    pickle.dump(dict_log, f)

print("Saved to:", filename)


[OsqpEigen::Solver::initSolver] Unable to setup the workspace.
[OsqpEigen::Solver::updateHessianMatrix] Unable to Initialize the solver.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.


ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.
Number of islands: 5
Topology: Unconnected
Migration type: point-to-point
Migrant handling policy: preserve
Status: busy

Islands summaries:

	#  Type                    Algo                                                        Prob                                  Size  Status  
	-------------------------------------------------------------------------------------------------------------------------------------------
	0  Multiprocessing island  CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy  <class '__main__.GainTuningProblem'>  10    busy    
	1  Multiprocessing island  CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy  <class '__main__.GainTuningProblem'>  10    busy    
	2  Multiprocessing island  CMA-ES: Covariance Matrix Adaptation Evolutionary St

ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


[OsqpEigen::Solver::initSolver] Unable to setup the workspace.
[OsqpEigen::Solver::updateHessianMatrix] Unable to Initialize the solver.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.


ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


...

[OsqpEigen::Solver::initSolver] Unable to setup the workspace.
[OsqpEigen::Solver::updateHessianMatrix] Unable to Initialize the solver.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.
[OsqpEigen::Solver::updateHessianMatrix] The solver has not been initialized.


ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


[OsqpEigen::Solver::initSolver] Unable to setup the workspace.
[OsqpEigen::Solver::updateHessianMatrix] Unable to Initialize the solver.
Function solver (0x563308d15cf0)
Input 0 (x0): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Exit condition -- generations = 10
Number of islands: 5
Topology: Unconnected
Migration type: point-to-point
Migrant handling policy: preserve
Status: idle

Islands summaries:

	#  Type                    Algo                                                        Prob                                  Size  Status  
	-------------------------------------------------------------------------------------------------------------------------------------------
	0  Multiprocessing island  CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy  <class '__main__.GainTuningProblem'>  10    idle    
	1  Multiprocessing island  CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy  <class '__main__.GainTuningProblem'>  10    idle    
	2  Multiprocessing island  CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy  <class '__main__.GainTuningProblem'>  10    idle    
	3  Multiprocessing island  CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy  <class '__main__.GainTuningPro

NameError: name 'datetime' is not defined

In [17]:
res

[array([-4.241]),
 array([-2.311]),
 array([-4.971]),
 array([-4.741]),
 array([-8.241])]

In [18]:
x

[array([7.06598416e-01, 0.00000000e+00, 7.33009989e+02, 6.54017050e+02,
        1.00000000e+03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+03, 0.00000000e+00, 5.42224617e+02,
        1.00000000e+03, 2.59819575e+02, 5.83856483e+01, 6.53166455e+02,
        7.76084922e+02, 1.00000000e+03, 0.00000000e+00, 5.37609071e+02,
        8.71753667e+01, 0.00000000e+00, 4.35173587e+02, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+03, 0.00000000e+00, 6.31096577e+01]),
 array([ 346.64771213,  352.0774424 ,  250.80467977,  523.62934927,
         158.8448143 ,    0.        ,  462.62828118,   78.38700752,
         490.58485793,  698.20860425, 1000.        , 1000.        ,
           0.        ,  240.19204056,  382.73663626,  228.96315863,
         504.38556018,  865.96246432,  897.63664672,  508.22497123,
         202.26851634,    0.        ,  757.99420444,  207.17623178,
           0.        ,  441.71278962,    0.        ,   14.31871712]),
 array([ 343.131

In [24]:
dict_log = {}
dict_log["res"] = res
dict_log["x"] = x

filename = "./{}.pkl".format(
    datetime.now().strftime("%d%m%Y%H%M%S")
)
with open(filename, "wb") as f:
    pickle.dump(dict_log, f)

print("Saved to:", filename)


Saved to: ./13122023105748.pkl


In [26]:
np.argmin(res)

4

In [28]:
cost_function(x[4])

0